In [1]:
from BlackjackEnv import BlackjackEnv
import numpy as np
import argparse
from agent import DQN_Agent

import os
from copy import deepcopy
import numpy as np
import torch.optim
from xuance import get_arguments
from xuance.common import space2shape
from xuance.environment import make_envs
from xuance.torch.utils.operations import set_seed
from xuance.torch.utils import ActivationFunctions

pygame 2.1.0 (SDL 2.0.16, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def parse_args():
    parser = argparse.ArgumentParser("Run a demo.")
    parser.add_argument("--method", type=str, default="dqn")
    parser.add_argument("--env", type=str, default="BlackjackEnv")
    parser.add_argument("--env-id", type=str, default="blackjack-v0")
    parser.add_argument("--test", type=int, default=0)
    parser.add_argument("--benchmark", type=int, default=1)
    parser.add_argument("--device", type=str, default="cuda:0")
    parser.add_argument("--config", type=str, default="./configs/test_blackjack.yaml")
    return parser.parse_args()

In [3]:
def sample(env, timestep_max, number,gamma,V_table):
    ''' 采样函数,策略Pi,限制最长时间步timestep_max,总共采样序列数number '''
    episodes = []
    for i in range(number):
        episode = []
        timestep = 0
        obs,info=env.reset()
        # 当前状态为终止状态或者时间步太长时,一次采样结束
        obs=obs.astype(int)
        terminated, truncated = False, False

        # 在状态s下根据策略选择动作
        while not terminated and timestep <= timestep_max:
            a = eps_greedy(env.action_space, 0.99*(number-i)/number, V_table,obs)
            next_obs, reward, terminated, truncated, info = env.step(a)
            next_obs = next_obs.astype(int)
            V_table[int(obs[0][0])][int(obs[0][1])][int(obs[0][2])] +=gamma**timestep * reward
            episode.append([obs, a, reward, next_obs])  # 把(obs, a, reward, next_obs)元组放入序列中
            obs = next_obs  # s_next变成当前状态,开始接下来的循环
            timestep += 1
        episodes.append(episode)
    return episodes

In [4]:
def MC(episodes, V_table, gamma,l_rate):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):  #一个序列从后往前计算
            [obs, a, r, obs_next] = episode[i]
            index_value=V_table[int(obs[0][0])][int(obs[0][1])][int(obs[0][2])]
            G = r + gamma * G
            V_table[int(obs[0][0])][int(obs[0][1])][int(obs[0][2])] = index_value+ (G - index_value) * l_rate

In [5]:
def eps_greedy(action_space,epsilon,V_table,obs)->int:#返回一个动作
    obs=obs[0].astype(int)
    if np.random.rand() < epsilon:
        return action_space.sample()
    else:
        #如果动作为0，则在+1~+10中取所有后选状态的平均值;动作为1则取当前状态的值
        for action in range(action_space.n):
            if action==0:#代表的意思是要所有的（在2-10以内）无A和有A，两类加起来平均
                action_0=np.sum(V_table[obs[0]][obs[1]+2:obs[1]+10][0])+V_table[obs[0]][obs[1]][1]
                action_0/=10

            elif action==1:
                action_1=V_table[obs[0]][obs[1]][obs[2]]
        if action_0> action_1:
            return 0
        return 1

In [6]:

def MC_test(V_table,env,number):
    obs,info = env.reset()
    obs = obs.astype(int)

    res=[]
    episodes=[]
    for i in range(number):
        r=0
        episode=[]
        terminated, truncated = False, False
        while not terminated :
            a = eps_greedy(env.action_space, 0, V_table, obs)
            next_obs, reward, terminated, truncated, info = env.step(a)
            next_obs = next_obs.astype(int)
            episode.append([obs])
            obs = next_obs  # s_next变成当前状态,开始接下来的循环
            r+=reward
        res.append(r)
        episodes.append(episode)
        obs,info=env.reset()
    return res,episodes

In [7]:

def run(args):
    agent_name = args.agent  # get the name of Agent.
    set_seed(args.seed)  # set random seed.

    # prepare directories for results
    args.model_dir = os.path.join(os.getcwd(), args.model_dir, args.env_id)  # the path for saved model.
    args.log_dir = os.path.join(args.log_dir, args.env_id)  # the path for logger file.

    # build environments
    env = make_envs(args)  # create simulation environments
    args.observation_space = env.observation_space  # get observation space
    args.action_space = env.action_space  # get action space
    gamma = 1.0  # 没有折扣
    l_rate=0.01
    V_table=np.zeros((12,30,2)) #一个12*30*2的表格来存储状态价值
    # 采样5次,每个序列最长不超过10步
    episodes = sample(env, 20, 5000, gamma,V_table)
    print('第一条序列\n', episodes[0])
    MC(episodes, V_table, gamma,l_rate=l_rate)
    res,ep=MC_test(V_table,env,10)
    print(res)
    print(ep,end='\n')
if __name__ == '__main__':
    parser = parse_args()
    args = get_arguments(method=parser.method,
                         env=parser.env,
                         env_id=parser.env_id,
                         config_path=parser.config,
                         parser_args=parser)
    run(args)


usage: Run a demo. [-h] [--method METHOD] [--env ENV] [--env-id ENV_ID]
                   [--test TEST] [--benchmark BENCHMARK] [--device DEVICE]
                   [--config CONFIG]
Run a demo.: error: unrecognized arguments: -f C:\Users\20230921a\AppData\Roaming\jupyter\runtime\kernel-33c4531a-3426-44c9-877f-575b96a7b322.json


SystemExit: 2

C:\Users\20230921a\.conda\envs\xuance\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
